In [24]:
import tensorflow as tf
import tensorflow
from tensorflow import keras
from tensorflow.keras.datasets import mnist,fashion_mnist
from keras import layers


In [25]:
(x_train,y_train),(x_test,y_test) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [26]:
x_train = x_train.reshape(-1,28,28,1).astype("float32")/255.0
x_test = x_test.reshape(-1,28,28,1).astype("float32")/255.0

In [27]:
class CNNBlock(layers.Layer):
  def __init__(self,out_chanels,kernel_size=3):
    super(CNNBlock,self).__init__()
    self.conv = layers.Conv2D(out_chanels,kernel_size,padding='same')
    self.bn = layers.BatchNormalization()

  def call(self,input_tensor, training=False):
    x = self.conv(input_tensor)
    x = self.bn(x,training=training)
    x = tf.nn.relu(x)
    return x

In [28]:
class VGGBlock(layers.Layer):
  def __init__(self,channels,extra_cnn_block = False):
    super(VGGBlock,self).__init__()
    self.cnn1 = CNNBlock(channels[0])
    self.cnn2 = CNNBlock(channels[0])
    self.extra_cnn_block = extra_cnn_block
    if self.extra_cnn_block:
      self.cnn3 = CNNBlock(channels[0])
    self.pooling = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))

  def call(self,input_tensor,training=False):
    x = self.cnn1(input_tensor,training=training)
    x = self.cnn2(x,training=training)
    if self.extra_cnn_block:
      x =  self.cnn3(x,training=training)
    x = self.pooling(x)
    return x

In [29]:
class VGG_DenseBlock(layers.Layer):
  def __init__(self,units):
    super(VGG_DenseBlock,self).__init__()
    self.dense = layers.Dense(units)
    self.drop = layers.Dropout(0.5)
    
  def call(self,input_tensor,training=False):
    x = self.dense(input_tensor,training=training)
    x = self.drop(x,training=training)
    x = tf.nn.relu(x)
    return x

In [30]:
class VGG_Like(keras.Model):
  def __init__(self,input_s,num_classes = 10):
    super(VGG_Like,self).__init__()
    self.input_s = input_s
    self.block1 = VGGBlock([64])
    self.block2 = VGGBlock([128])
    # self.block3 = VGGBlock([256],True)
    # self.block4 = VGGBlock([512],True)
    # self.block5 = VGGBlock([8],True)
    self.flatten = layers.Flatten()
    self.dense1 = VGG_DenseBlock(4096)
    self.dense2 = VGG_DenseBlock(4096)
    self.classifier = layers.Dense(num_classes)

  def call(self,input_tensor,training=False):
    x = self.block1(input_tensor,training=training)
    x = self.block2(x,training=training)
    # x = self.block3(x,training=training)
    # x = self.block4(x,training=training)
    # x = self.block5(x,training=training)
    x = self.flatten(x,training=training)
    x = self.dense1(x,training=training)
    x = self.dense2(x,training=training)
    out =  self.classifier(x)
    return out

  def model(self):
    x = keras.Input(shape=self.input_s)
    return keras.Model(inputs=[x],outputs=self.call(x))


In [31]:
model = VGG_Like(input_s=(28,28,1))

In [32]:
model.compile(
    optimizer = keras.optimizers.Adam(),
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics= ['accuracy'] 
)

In [33]:
model.model().summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 vgg_block_6 (VGGBlock)      (None, 14, 14, 64)        38080     
                                                                 
 vgg_block_7 (VGGBlock)      (None, 7, 7, 128)         222464    
                                                                 
 flatten_3 (Flatten)         (None, 6272)              0         
                                                                 
 vgg__dense_block_6 (VGG_Den  (None, 4096)             25694208  
 seBlock)                                                        
                                                                 
 vgg__dense_block_7 (VGG_Den  (None, 4096)             16781312  
 seBlock)                                                  

In [34]:
model.fit(x_train,y_train,batch_size=64,epochs=3,verbose=2)
model.evaluate(x_test,y_test,batch_size=64,verbose=2)

Epoch 1/3
938/938 - 20s - loss: 1.0075 - accuracy: 0.7268 - 20s/epoch - 22ms/step
Epoch 2/3
938/938 - 19s - loss: 0.5646 - accuracy: 0.8044 - 19s/epoch - 20ms/step
Epoch 3/3
938/938 - 19s - loss: 0.4642 - accuracy: 0.8394 - 19s/epoch - 21ms/step
157/157 - 1s - loss: 0.3529 - accuracy: 0.8867 - 970ms/epoch - 6ms/step


[0.35292866826057434, 0.8866999745368958]